### Imports

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
#import matplotlib.pyplot as plt
#from feature_baggingV2 import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import datetime
from sklearn.preprocessing import RobustScaler
#from keras.models import Sequential
#from keras.layers import LSTM, Dense
import labolibrary as labo

import os, sys, gc, time, warnings, pickle, psutil, random

import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from sktime.clustering.k_means import TimeSeriesKMeans
from sktime.clustering.utils.plotting._plot_partitions import plot_cluster_algorithm
from sktime.datasets import load_arrow_head

### Función para escalar/desescalar y métrica

In [ ]:
#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Leer datos
df_final = pd.read_parquet(DATOS_DIR+'FE_09_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [5]:


# Ensure periodo is not the index for fitting the model
df_final.reset_index(inplace=True)

# Handle missing values by forward fill and then backward fill
df_final['tn'] = df_final.groupby(['product_id', 'customer_id'])['tn'].apply(lambda group: group.ffill().bfill())

# Create a pivot table to reshape the dataframe
pivot_df = df_final.pivot(index='periodo', columns=['product_id', 'customer_id'], values='tn')

# Fill any remaining missing values in the pivot table (if any)
pivot_df.fillna(method='ffill', inplace=True)
pivot_df.fillna(method='bfill', inplace=True)

# Transpose and reshape the pivot table to get the right format for clustering
ts_data = pivot_df.T.values.reshape(pivot_df.shape[1], pivot_df.shape[0], 1)

# Initialize the TimeSeriesKMeans model
model = TimeSeriesKMeans(n_clusters=10, metric="dtw",
                         init_algorithm='kmeans++', n_init=2, max_iter=200, tol=10000,
                         verbose=True, random_state=42)

# Fit the model and predict clusters
clusters_dtw = model.fit_predict(ts_data)

# Create a DataFrame for the clusters with the same index as the pivot table columns
cluster_df = pd.DataFrame({'dtw_cluster': clusters_dtw}, index=pivot_df.columns)

# Reset the index of the original dataframe to align for merging
df_final.set_index(['product_id', 'customer_id'], inplace=True)

# Merge the clusters back to the original dataframe
df_final = df_final.merge(cluster_df, left_index=True, right_index=True, how='left')

# Reset the index to its original state
df_final.reset_index(inplace=True)

df_final.head()


TypeError: incompatible index of inserted column with frame index